In [1]:
#import solcx
#solcx.install_solc('0.8.28')

In [2]:
#%pip install ape-solidity

In [3]:
#%pip install eth-tester

In [4]:
import shutil
import time
import pandas as pd
import random

In [5]:
from ape import project
from ape import Project
from ape import networks

from ape import chain
from ape import accounts

from ape.logging import logger, LogLevel


logger.set_level(LogLevel.WARNING)
sender = accounts.test_accounts[0]

In [6]:
#orig = Project('.')
#a = orig.get_contract('PIController.sol')
try:
    shutil.rmtree("src/.ipynb_checkpoints")
except:
    print("can't delete")
    pass
old_pi = project.get_contract("PIController") 

can't delete


In [7]:
new_pi = project.get_contract("PIControllerNew") 

In [8]:
rpc = 'foundry'

In [9]:
kp = 222002205862
ki = 25000

kp = 222002205862000000
ki = 250000000000000000

coBias = 0
perSecondIntegralLeak = 999997208243937652252849536 #1% per hour
#perSecondIntegralLeak = int(10**27)
outputUpperBound = 18640000000000000000
outputLowerBound = -51034000000000000000

importedState = [0,0,0]
with networks.ethereum.local.use_provider(rpc):
    old_pi_c = old_pi.deploy(
            b'test control variable',
            kp,
            ki,
            coBias,
            perSecondIntegralLeak,
            outputUpperBound,
            outputLowerBound,
            importedState,
            sender=sender
          )

ChainError: '0xa1001a55aad9c6487d189993ff04ee9400ed59efcb0cf80c85ea0770d2dcefc3' did not create a contract.

In [ ]:
with networks.ethereum.local.use_provider(rpc):
    new_pi_c = new_pi.deploy(
            b'test control variable',
            kp,
            ki,
            coBias,
            perSecondIntegralLeak,
            outputUpperBound,
            outputLowerBound,
            importedState,
            sender=sender
          )

In [ ]:
with networks.ethereum.local.use_provider(rpc):
    new_pi_c.modifyParametersAddr('seedProposer',  sender, sender=sender)

In [ ]:
# not workinging, so commented out seedProposer requirement
#with networks.ethereum.local.use_provider("test"):
#    old_pi_c.modifyParameters(b"seedProposer",  sender, sender=sender)

In [ ]:
random.randint(1, 2**256)

In [ ]:
n = 20
data = []
for _ in range(n):
    data.append([random.randint(1, 2**256), random.randint(1, 2**256), random.randint(1, 2**256)])

In [ ]:
n = len(data)
old_values = []
new_values = []

In [ ]:
with networks.ethereum.local.use_provider(rpc):
    chain.provider.auto_mine = False
    for a,b,c in data:
        val = old_pi_c.rpower(a, b, c)
        old_values.append(val)


In [ ]:
with networks.ethereum.local.use_provider(rpc):
    chain.provider.auto_mine = False
    for a,b,c in data:
        val = new_pi_c.my_exp_uint256_external(a, b, c)
        new_values.append(val)

In [ ]:
df_new = pd.DataFrame(new_values, columns = ['output_new', 'p_output_new', 'i_output_new', 'ts_new'])
df_old = pd.DataFrame(old_values, columns = ['output_old', 'p_output_old', 'i_output_old', 'ts_old'])
#df = pd.DataFrame(values, columns = ['error_integral', 'error'])

In [ ]:
df_new.to_csv('new.csv.gz', index=False)
df_old.to_csv('old.csv.gz', index=False)

In [ ]:
all_df = pd.concat((df_new, df_old), axis=1)

In [ ]:
all_df.head(10)

In [ ]:
all_df['output_new'].equals(all_df['output_old'])

In [ ]:
all_df['p_output_new'].equals(all_df['p_output_old'])

In [ ]:
all_df['i_output_new'].equals(all_df['i_output_old'])